# Accelerometer Data QA & Analysis

- This Notebook is primarily for assessing the quality and validity of the accelerometer generator data.
- As the formulas used are just my best interpretation of the domain concepts and assume various ideal conditions, I figured it would be best to see how does this simulated data compare to real-world data using basic statistical analysis
- This notebook will ignore the data generated and pushed to S3 because overall this project is just for learning purposes and the fewer (though very minor) Cloud usage-incurred costs the better.

In [ ]:
# Imports
import time
from typing import Any
from datetime import datetime, timezone, timedelta

%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
from scipy.fft import fft, fftfreq

from generators.accelerometer import GenerateDataParams, generate_data

## Generate Data
- Generate partitioned data spread across several hours. The goal is to create a significant amount of data for testing purposes
- Some assumptions here
  - The quadriped is walking non-stop for the alloted time. Too messy to get into the business of simulating the robot coming to a stop and resuming walking
  - For now, starting parameters (like amplitude, base, and phase values) will be random but within some range that feels reasonable. They will also be constant for the entire data set for simplicity's sake.

In [ ]:
# Create Accelerometer parameters
start_ts = datetime.now(timezone.utc)
hours = 6  # Modify this as needed
sample_frequency = 100 # Modify this as needed
total_time = hours * 60 * 60

params = GenerateDataParams()

In [ ]:
# Generate data and store in DataFrame
data_gen_start_ts = time.perf_counter()
data_df = generate_data(frequency=sample_frequency, total_time=total_time, start_time=start_ts, params=params)
data_gen_duration = time.perf_counter() - data_gen_start_ts
print(f"Time to generate {len(data_df)} records: {data_gen_duration:.4f} seconds")
print("\nDataFrame Info:")
data_df.info()

## Visual Data Analysis
- Just a simple visual confirmation that the data is following the desired sinusoidal pattern with acceptable levels of noise


In [ ]:
# Visual Data Analysis
if "data_df" in locals() and not data_df.empty:
    subset_df = data_df.iloc[:sample_frequency * 10]  # Just the first 10 seconds of data
    plt.figure(figsize=(15,7))

    plt.plot(subset_df["timestamp"], subset_df["accel_x"], label="Accel X", color="red", linewidth=1.5)
    plt.plot(subset_df["timestamp"], subset_df["accel_y"], label="Accel Y", color="blue", linewidth=1.5)
    plt.plot(subset_df["timestamp"], subset_df["accel_z"], label="Accel Z", color="green", linewidth=1.5)

    plt.xlabel("Time", fontsize=14)
    plt.ylabel("Acceleration (m/s^2)", fontsize=14)
    plt.title("Simulated Accelerometer Data Over Time", fontsize=16)
    plt.legend(loc="upper right", fontsize=12)
    plt.xticks(rotation=45, ha="right")
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
    plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator(minticks=5, maxticks=10))
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.tight_layout()
    plt.show()
else:
    print("DataFrame 'data_df' is not defined or is empty. Please generate data first.")

## Understanding the Visual Representation of Acceleration Vectors over Time
- Assessing a subset of the data, it's clear to see that each of the acceleration vector's values over time achieve the attended goals of data simulator:
  - All vectors follow a periodic, sinusoidal pattern aligning with the Simple Harmonic Motion (SHM) approximation used to model the oscillations (due to gait, bounce, sway, pitch, and roll) of a quadruped robot walking
  - All vectors observe a consistent amount of simulated noise in the data (the "shakiness" of each of the lines), mimicking imperfections in real-world sensor readings
  - All seem within tolerable ranges given the GenerateDataParams (in this case the default values for the parameters)
  - The Z-axis acceleration values center around the constant gravity vector value of 9.8 m/s^2, and the X and Y-axis accelerations are centered around very small base_pitch and base_roll values.
- Assuming several ideal conditions (traveling in one direction, walking on a flat plane, walking at constant speed, SHM, etc.) this data at first glance seems to provide a good qualitative representation of the readings of an accelerometer contained within a walking quadruped robot.

## Frequency Domain Analysis (FFT)
- A very cool 3blue1brown video [here](https://www.youtube.com/watch?v=spUNpyF58BY) on Fourier Transforms
- Motivation is to identify the dominent frequencies which should be the `gait_frequency` (and potentially its half for sway. See accelereometer.generate_data code) and its harmonics
- This concept was (still kind of is) rather foreign to me so including several comments for later reference.

In [ ]:
if "data_df" in locals() and not data_df.empty:
    fft_subset_duration = 10  # seconds
    fft_samples = sample_frequency * fft_subset_duration
    # Calculate Fourier Transform (FT) for just a small subset (first 10 seconds) of data.
    fft_df = data_df.iloc[:fft_samples].copy()
    if len(fft_df) >= 2:
        T = 1.0 / sample_frequency  # Sample spacing, time in seconds between data records
        N = len(fft_df)  # Number of sample points for FFT

        signals = {
            "Accel X": fft_df["accel_x"].values,
            "Accel Y": fft_df["accel_y"].values,
            "Accel Z": fft_df["accel_z"].values
        }

        plt.figure(figsize=(15, 12))
        plot_index = 1
        for name, data in signals.items():
            if N > 0 and len(data) == N:
                # Get dataframe of FT complex numbers for input signals
                yf = fft(data)
                # Get the frequency bins that these complex numbers correspond to.
                # Only care about the positive values here, hence filtering only to [:N//2] results
                xf = fftfreq(N, T)[:N//2]

                plt.subplot(3, 1, plot_index)
                # 2.0/N - normalization factor where we multiply by 2 purely for visualization purposes,
                # and divide by N to scale the amplitude to be independent of the signal length
                # Complex numbers are kinda hard to visualize, so we take the absolute value
                # Again, only want the positive frequencies so we do yf[0:N//2]
                plt.plot(xf, 2.0/N * np.abs(yf[0:N//2]))
                plt.title(f'Frequency Spectrum of {name}')
                plt.xlabel('Frequency (Hz)')
                plt.ylabel('Amplitude')
                # Highlight expected gait frequency (from params object)
                if params and hasattr(params, 'gait_frequency'):
                    plt.axvline(params.gait_frequency, color='r', linestyle='--', label=f'Gait Freq: {params.gait_frequency:.2f} Hz')
                    # For Accel Y, sway might be half the gait frequency based on our model
                    if name == 'Accel Y' and (params.gait_frequency / 2) > 0:
                         plt.axvline(params.gait_frequency / 2, color='m', linestyle=':', label=f'Sway Freq: {params.gait_frequency/2:.2f} Hz')
                plt.legend()
                plt.grid(True)
                plot_index += 1
            else:
                 print(f"Signal data for {name} is empty or length mismatch, skipping FFT. N={N}, len(data)={len(data)}")

        plt.tight_layout()
        plt.show()
    else:
        print("Not enough samples in subset for FFT analysis.")
else:
    print("Dataframe 'data_df' is not defined or is empty for FFT analysis.")

## Understanding the Fourier Transform Plots
- The X axis represents all the different frequencies that the FFT analyzed
- The Y axis represents the "magnitude" of each frequency component found in the input signal. The taller the value, the more the input signal contained strong sine wave-like oscillation at that particular frequency.
- Essentially this process is asking the question of "How much do these signals (X, Y, Z acceleration) look like a 0.1 Hz sine wave? How about a 0.2 Hz sine wave? 2 Hz (our gait frequency)?" and the peaks represent how strongly the signals represent those frequencies.
- Frequency Spectrum of X-Axis Acceleration:
  - The peak at 2.0 Hz strongly suggests the X-axis acceleration is influenced by the front-to-back pitching motion
  - The peak near 0 Hz corresponds to the non-zero base_pitch value
- Frequency Spectrum of Y-Axis Acceleration:
  - Peak at 1.0 Hz: In the SHM model code, `omega_sway = 2 * np.pi * (gait_frequency / 2)` which means the sway frequency is half the gait frequency of 2.0 Hz. This corresponds to the side-to-side motion
  - Smaller Peak at 2.0 Hz: A harmonic. In a real-world scenario, This could becaues the pitching and rolling motion mighth have components that project onto the Y-axis of the sensor
- Frequency Spectrum of Z-Axis Acceleration:
  - Large peak at 0 Hz: Represents the avereage value of the signal, which in this case is the constant effect of gravity (+9,81 m/s^2)
  - Peak at 2.0 Hz: This corresponds with the gait frequency and involves the "bounce" component.
- Ultimately, the FFT analysis confirms that the intended frequencies are indeed present in the output acceleration values and that there are not unexpected dominant frequencies present. The relative strengths (amplitudes) of the signals also tells us how much of a roll each component plays.